In [ ]:
import os
from pathlib import Path
import math
from joblib import Parallel, delayed
from tqdm import tqdm
import shutil
import itertools
import cv2
import time
from utils import ScaleCategory, load_dataset_dir

#import matplotlib.pyplot as plt
#%matplotlib inline

data_dir = Path.cwd() / "data_scale"
dataset_dir = data_dir / "dataset"
inference_dir = data_dir / "inference"

In [ ]:
shutil.rmtree(inference_dir, ignore_errors=True)
time.sleep(1)
inference_dir.mkdir(exist_ok=True)

In [ ]:
dataset_files = load_dataset_dir(dataset_dir, ScaleCategory)
dataset_files = [p.stem for p, _ in dataset_files]

still_valid_images_dir = dataset_dir / ScaleCategory.valid.name
parameters = list(still_valid_images_dir.glob("*.png"))
print("candidate files:", len(parameters))

parameters = [(p, p.stem + "_") for p in parameters]
parameters = [(p, inference_dir / (name + ".png")) for p, name in parameters if not name in dataset_files]

print("resize files:", len(parameters))

In [ ]:
scale_factor = 0.2

def process(param):
    src_path, dst_path = param

    image = cv2.imread(str(src_path), cv2.IMREAD_COLOR)
    image_h, image_w, _ = image.shape

    trim_w = math.ceil(image_w * scale_factor * 0.5)
    trim_h = math.ceil(image_h * scale_factor * 0.5)

    crop_image = image[trim_h:image_h-trim_h-1, trim_w:image_w-trim_w-1, :]
    crop_image = cv2.resize(crop_image, (image_w, image_h), interpolation = cv2.INTER_CUBIC)

    if (False):
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.show()
        plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
        plt.show()

    cv2.imwrite(str(dst_path), crop_image)

#[process(params) for params in tqdm(parameters)]
Parallel(n_jobs=-1, verbose=10)([delayed(process)(params) for params in parameters])
None